In [ ]:
import pandas as pd
import numpy as np
import talib  # For technical indicators

def calculate_macd_mtf(df, close_column='Close', fast_length=12, slow_length=26, signal_length=9, res='D'):  # res is the timeframe
    """Calculates the Multi-Timeframe MACD indicator.

    Args:
        df: Pandas DataFrame with a 'Close' column (or specified).
        close_column: Name of the column containing closing prices.
        fast_length: Length of the fast EMA.
        slow_length: Length of the slow EMA.
        signal_length: Length of the signal SMA.
        res: Timeframe for the calculation (e.g., 'D', 'H', 'M').  If None, uses df's timeframe.

    Returns:
        DataFrame with MACD, Signal, and Histogram values.
    """

    # Ensure the DataFrame has a DatetimeIndex if res is specified
    if res is not None and not pd.api.types.is_datetime64_any_dtype(df.index):
        try:
            df.index = pd.to_datetime(df.index)
        except Exception as e:  # Handle potential conversion errors
            print(f"Error converting index to DateTimeIndex: {e}")
            return None
        df = df.resample(res).agg({close_column: 'last'}) #Resample the dataframe

    source = df[close_column]

    fast_ma = talib.EMA(source, fast_length)
    slow_ma = talib.EMA(source, slow_length)
    macd = fast_ma - slow_ma
    signal = talib.SMA(macd, signal_length)
    hist = macd - signal

    df['MACD'] = macd
    df['Signal'] = signal
    df['Histogram'] = hist

    # Histogram color logic (vectorized for efficiency)
    hist_a_up = (df['Histogram'] > df['Histogram'].shift(1)) & (df['Histogram'] > 0)
    hist_a_down = (df['Histogram'] < df['Histogram'].shift(1)) & (df['Histogram'] > 0)
    hist_b_down = (df['Histogram'] < df['Histogram'].shift(1)) & (df['Histogram'] <= 0)
    hist_b_up = (df['Histogram'] > df['Histogram'].shift(1)) & (df['Histogram'] <= 0)

    df['Hist_Color'] = np.select(
        [hist_a_up, hist_a_down, hist_b_down, hist_b_up],
        ['aqua', 'blue', 'red', 'maroon'],
        default='gray'
    )

    # MACD color logic
    macd_is_above = df['MACD'] >= df['Signal']
    df['Macd_Color'] = np.where(macd_is_above, 'lime', 'red')
    df['Signal_Color'] = np.where(macd_is_above, 'yellow', 'yellow')

    return df

# Sample DataFrame (replace with your actual data)
data = pd.DataFrame({
    'Close': [10, 12, 15, 13, 16, 18, 15, 17, 20, 18],
}, index=pd.to_datetime(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12']))

df_result = calculate_macd_mtf(data, res='D')  # Calculate daily MACD

if df_result is not None:
    print(df_result)

    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 6))
    plt.plot(df_result['MACD'], label='MACD', color="red")#df_result['Macd_Color']
    plt.plot(df_result['Signal'], label='Signal', color="green")#df_result['Signal_Color']
    plt.bar(df_result.index, df_result['Histogram'], color=df_result['Hist_Color'], label='Histogram', width=1) #Histogram Plotting
    plt.legend(loc='best')
    plt.title('MACD MTF')
    plt.grid(True)
    plt.show()
else:
    print("MACD calculation failed. Check your DataFrame and parameters.")


# Example with a different timeframe (e.g., hourly)
df_hourly = calculate_macd_mtf(data, res='H')
if df_hourly is not None:
    print(df_hourly)
else:
    print("MACD calculation failed. Check your DataFrame and parameters.")